In [7]:
import pandas as pd
from pandas import DataFrame
import numpy as np

### 处理丢失数据
- 有两种丢失数据：
    - None
    - np.nan(NaN)

- 两种丢失数据的区别

In [8]:
type(None)

NoneType

In [9]:
type(np.nan)

float

- 为什么在数据分析中需要用到的是浮点类型的空而不是对象类型？
    - 数据分析中会常常使用某些形式的运算来处理原始数据，如果原数数据中的空值为NAN的形式，则不会干扰或者中断运算。
    - NAN可以参与运算的
    - None是不可以参与运算

In [10]:
np.nan + 1

nan

In [11]:
None + 1

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

- 在pandas中如果遇到了None形式的空值则pandas会将其强转成NAN的形式。

In [13]:
df = DataFrame(data=np.random.randint(0,100,size=(7,5)))
df.iloc[2,3] = None
df.iloc[4,2] = np.nan
df.iloc[5,4] = None
df

,0,1,2,3,4
0,98,2,54.0,30.0,73.0
1,12,16,42.0,78.0,19.0
2,86,59,15.0,NaN,89.0
3,47,30,20.0,75.0,48.0
4,87,63,NaN,62.0,21.0
5,34,69,58.0,49.0,NaN
6,94,12,56.0,24.0,56.0


### pandas处理空值操作
- isnull
- notnull
- any
- all
- dropna
- fillna

- 方式1：对空值进行过滤（删除空所在的行数据）
    - 技术：isnull，notnull，any，all

In [ ]:
df.isnull()

,0,1,2,3,4
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,True,False
3,False,False,False,False,False
4,False,False,True,False,False
5,False,False,False,False,True
6,False,False,False,False,False


In [ ]:
#哪些行中有空值
#any(axis=1)检测哪些行中存有空值
df.isnull().any(axis=1) #any会作用isnull返回结果的每一行
#true对应的行就是存有缺失数据的行

0    False
1    False
2     True
3    False
4     True
5     True
6    False
dtype: bool

In [ ]:
df.notnull()

,0,1,2,3,4
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,False,True
3,True,True,True,True,True
4,True,True,False,True,True
5,True,True,True,True,False
6,True,True,True,True,True


In [ ]:
df.notnull().all(axis=1)

0     True
1     True
2    False
3     True
4    False
5    False
6     True
dtype: bool

In [ ]:
#将布尔值作为源数据的行索引
df.loc[df.notnull().all(axis=1)]

,0,1,2,3,4
0,82,94,60.0,48.0,57.0
1,12,1,37.0,18.0,61.0
3,60,41,29.0,39.0,5.0
6,68,41,76.0,92.0,62.0


In [ ]:
#获取空对应的行数据
df.loc[df.isnull().any(axis=1)]
#获取空对应行数据的行索引
indexs = df.loc[df.isnull().any(axis=1)].index
indexs

Int64Index([2, 4, 5], dtype='int64')

In [ ]:
df.drop(labels=indexs,axis=0)

,0,1,2,3,4
0,82,94,60.0,48.0,57.0
1,12,1,37.0,18.0,61.0
3,60,41,29.0,39.0,5.0
6,68,41,76.0,92.0,62.0


- 方式2：
    - dropna：可以直接将缺失的行或者列进行删除

In [14]:
df.dropna(axis=0)

,0,1,2,3,4
0,98,2,54.0,30.0,73.0
1,12,16,42.0,78.0,19.0
3,47,30,20.0,75.0,48.0
6,94,12,56.0,24.0,56.0


- 对缺失值进行覆盖
    - fillna

In [17]:
df.fillna(value=999) #使用指定值将源数据中所有的空值进行填充

,0,1,2,3,4
0,98,2,54.0,30.0,73.0
1,12,16,42.0,78.0,19.0
2,86,59,15.0,999.0,89.0
3,47,30,20.0,75.0,48.0
4,87,63,999.0,62.0,21.0
5,34,69,58.0,49.0,999.0
6,94,12,56.0,24.0,56.0


In [18]:
#使用空的近邻值进行填充
#method=ffill向前填充，bfill向后填充
df.fillna(axis=0,method='bfill')

,0,1,2,3,4
0,98,2,54.0,30.0,73.0
1,12,16,42.0,78.0,19.0
2,86,59,15.0,75.0,89.0
3,47,30,20.0,75.0,48.0
4,87,63,58.0,62.0,21.0
5,34,69,58.0,49.0,56.0
6,94,12,56.0,24.0,56.0


- 什么时候用dropna什么时候用fillna
    - 尽量使用dropna，如果删除成本比较高，则使用fillna

- 使用空值对应列的均值进行空值填充

In [20]:
for col in df.columns:
    #检测哪些列中存有空值
    if df[col].isnull().sum() > 0:#说明df[col]中存有空值
        mean_value = df[col].mean()
        df[col] = df[col].fillna(value=mean_value)
df

,0,1,2,3,4
0,98,2,54.000000,30.0,73.0
1,12,16,42.000000,78.0,19.0
2,86,59,15.000000,53.0,89.0
3,47,30,20.000000,75.0,48.0
4,87,63,40.833333,62.0,21.0
5,34,69,58.000000,49.0,51.0
6,94,12,56.000000,24.0,56.0
